In [1]:
import copy

In [2]:
file1 = open('input.txt', 'r')
lines = file1.readlines()

In [3]:
bricks = {}
positions = {}
layers = {}

angles = {0: (1, 0, 0), 1: (0, 1, 0), 2: (0, 0, 1)}

for i, line in enumerate(lines):
    name = f"b{i}"
    brick = list(tuple(map(int, b.strip().split(","))) for b in line.strip().split('~'))
    length = list(map(lambda x: abs(brick[0][x] - brick[1][x]), range(3)))
    nb_of_layer = max(length)
    angle = length.index(nb_of_layer)
    min_layer = min(list(map(lambda x: x[2], brick)))
    #print(f"brick {name}, {brick} has {nb_of_layer} layers and angle {angle} and min_layer {min_layer}")
    if nb_of_layer == 0:
        bricks[name] = brick[:-1]
        positions[brick[0][2]] = positions.get(brick[0][2], []) + [brick[0]]
        layers[min_layer] = layers.get(min_layer, []) + [name]
    elif nb_of_layer == 1:
        bricks[name] = brick
        for b in brick:
            positions[b[2]] = positions.get(b[2], []) + [b]
        layers[min_layer] = layers.get(min_layer, []) + [name]
    else:
        smallest = 0 if sum(brick[0]) < sum(brick[1]) else 1
        evolved_bick = brick
        layers[min_layer] = layers.get(min_layer, []) + [name]
        for i in range(nb_of_layer-1):
            new_point = evolved_bick[smallest]
            for j in range(i+1):
                new_point = tuple(map(lambda x: x[0] + x[1], zip(new_point, angles[angle])))
            evolved_bick.append(new_point)
        bricks[name] = evolved_bick
        for b in evolved_bick:
            positions[b[2]] = positions.get(b[2], []) + [b]


def lower_layer(layer, local_position, local_layers, local_bricks):
    moved = False
    if layer not in local_layers:
        return local_position, local_layers, local_bricks, moved
    new_layer = {}
    for i in local_layers[layer]:
        brick = local_bricks[i]
        for b in brick:
            local_position[b[2]].remove(b)
        bottom = False
        while not bottom:
            for b in brick:
                if (b[0], b[1], b[2]-1) in local_position.get(b[2]-1, []) or b[2] == 1:
                    bottom = True
            if not bottom:
                brick = [tuple(map(lambda x: x[0] - x[1], zip(new_point, angles[2]))) for new_point in brick]
                moved = True
        for b in brick:
            local_position[b[2]] = local_position.get(b[2], []) + [b]
        min_layer = min(list(map(lambda x: x[2], brick)))
        new_layer[min_layer] = new_layer.get(min_layer, []) + [i]
        local_bricks[i] = brick
    local_layers[layer] = []
    local_layers = { key:local_layers.get(key,[])+new_layer.get(key,[]) for key in set(list(local_layers.keys())+list(new_layer.keys())) }
    return local_position, local_layers, local_bricks, moved


def remove(brick_name, layer_number):
    local_position = copy.deepcopy(positions)
    local_layers = copy.deepcopy(layers)
    local_bricks = copy.deepcopy(bricks)
    brick = local_bricks[brick_name]
    for b in brick:
        local_position[b[2]].remove(b)
    local_layers[layer_number].remove(brick_name)
    local_bricks.pop(brick_name)
    top = max(list(map(lambda x: x[2], brick)))

    tmp_positions, tmp_layers, tmp_bricks, moved = lower_layer(layer_number+1, local_position, local_layers, local_bricks)

    if top == layer_number:
        return not moved
    if moved:
        return False
    tmp_positions, tmp_layers, tmp_bricks, moved_top = lower_layer(top+1, local_position, local_layers, local_bricks)


    return not moved and not moved_top



In [4]:
ever_moved = []
for i in range(1, max(layers.keys()) + 1):
    positions, layers, bricks, moved = lower_layer(i, positions, layers, bricks)
    ever_moved.append(moved)

In [5]:
counter_bricks = 0
bricks_chain = {}
for layer in layers:
    for brick in layers[layer]:
        if remove(brick, layer):
            counter_bricks += 1
print(f"Part1 : {counter_bricks}")

Part1 : 530


In [6]:
def lower_layer(layer, local_position, local_layers, local_bricks):
    moved = False
    dropped = set()
    if layer not in local_layers:
        return local_position, local_layers, local_bricks, moved, dropped
    new_layer = {}
    for i in local_layers[layer]:
        brick = local_bricks[i]
        for b in brick:
            local_position[b[2]].remove(b)
        bottom = False
        while not bottom:
            for b in brick:
                if (b[0], b[1], b[2]-1) in local_position.get(b[2]-1, []) or b[2] == 1:
                    bottom = True
            if not bottom:
                brick = [tuple(map(lambda x: x[0] - x[1], zip(new_point, angles[2]))) for new_point in brick]
                dropped.add(i)
                moved = True
        for b in brick:
            local_position[b[2]] = local_position.get(b[2], []) + [b]
        min_layer = min(list(map(lambda x: x[2], brick)))
        new_layer[min_layer] = new_layer.get(min_layer, []) + [i]
        local_bricks[i] = brick
    local_layers[layer] = []
    local_layers = { key:local_layers.get(key,[])+new_layer.get(key,[]) for key in set(list(local_layers.keys())+list(new_layer.keys())) }
    return local_position, local_layers, local_bricks, moved, dropped

def remove(brick_name, layer_number):
    local_position = copy.deepcopy(positions)
    local_layers = copy.deepcopy(layers)
    local_bricks = copy.deepcopy(bricks)
    brick = local_bricks[brick_name]
    all_bricks = set()
    for b in brick:
        local_position[b[2]].remove(b)
    local_layers[layer_number].remove(brick_name)
    local_bricks.pop(brick_name)
    top = max(list(map(lambda x: x[2], brick)))

    local_position, local_layers, local_bricks, moved, dropped = lower_layer(layer_number+1, local_position, local_layers, local_bricks)
    all_bricks = all_bricks.union(dropped)
    if top == layer_number and not moved:
        return not moved, all_bricks

    for layer in layers:
        if layer <= layer_number + 1:
            continue
        local_position, local_layers, local_bricks, moved_top, dropped = lower_layer(layer, local_position, local_layers, local_bricks)
        all_bricks = all_bricks.union(dropped)


    return not moved and not moved_top, all_bricks

total_sum = 0
bricks_chain = {}
for layer in layers:
    for brick in layers[layer]:
        moved, dropped = remove(brick, layer)
        #print(f"brick {brick} moved {moved} dropped {dropped}")
        total_sum += len(dropped)
print(f"Part1 : {total_sum}")

Part1 : 93292
